<a href="https://colab.research.google.com/github/sefnyn/python_study_group/blob/main/Task02a/barbara/SectorSurveyDashApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dash App

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
%cd /content/drive/My Drive/Dash_app/

In [ ]:
pip install dash

In [ ]:
pip install dash-bootstrap-components

In [ ]:
pip install pgeocode


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from dash import Dash, html, dash_table, dcc, callback, Output, Input
from dash_bootstrap_components.themes import BOOTSTRAP
import dash_bootstrap_components as dbc
import os
import geopandas as gpd
import pgeocode

In [ ]:
# reading in data:

allData = pd.read_excel('SectorSurveysData_updated.xlsx')
# gdf_sp = gpd.read_file('RGN_DEC_2023_EN_BFC.shp')
# gdf = gpd.read_file('RGN_DEC_2023_EN_BFC.shx')
# gdf_dbf = gpd.read_file('RGN_DEC_2023_EN_BFC.dbf')
gdf_regions = gpd.read_file('NUTS_Level_1_January_2018_FCB_in_the_United_Kingdom.shp')
gdf_regions

In [ ]:
gdf_regions['nuts118nm'] = gdf_regions['nuts118nm'].str.replace(' (England)', '')
gdf_regions

In [ ]:
countries= gdf_regions[(gdf_regions['nuts118nm'] == 'Northern Ireland') | (gdf_regions['nuts118nm'] == 'Scotland')].index
gdf_regions.drop(countries, inplace=True)
gdf_regions

In [ ]:
# more data wrangling
df = allData

mask = (df['TNA Region'] == 'Yorkshire')
df.loc[mask,'Postcode Region'] = df.loc[mask,'TNA Region']
df.loc[df["Postcode Region"] == "Yorkshire", "Postcode Region"] = "Yorkshire and The Humber"
df.loc[df["Postcode Region"] == "Greater London", "Postcode Region"] = "London"

# changing outlier value to the mean
df.loc[df['Visitors to your archive service'] >500000, 'Visitors to your archive service']  = allData['Visitors to your archive service'].mean()

df.rename(columns = {'Answer':'Open to the public'}, inplace = True)
# Update the 'Open to the public' column where Category is 'Local Authority' and the value is missing
# Question not asked to LA archives; all are open to the public
df.loc[(df['Category'] == 'Local Authority Archive') & (allData['Open to the public'].isna()), 'Open to the public'] = 'Yes'

In [ ]:
# Binning data
s_bins = [0.1, 1, 5, 10, 15, 20, 25, 30]
s_labels = ['0.1-1', '1.1-5', '5.1-10', '10.1-15', '15.1-20', '20.1-25', '25.1-30']
df['archivists grouped'] = pd.cut(df['Total number of qualified archivists'], bins=s_bins, labels=s_labels, include_lowest=True)
archivistBins = df['archivists grouped'].value_counts(sort=False).reset_index().rename(columns={"index": "archivists", 0: "count"})

f_bins = [1, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 500000, 1000000, 20000000]
f_labels = ['1-10,000', '10,001-20,000', '20,001-30,000', '30,001-40,000', '40,001-50,000', '50,001-60,000', '60,001-70,000', '70,001-80,000', '80,001-90,000', '90,001-100,000', '100,001-500,000', '500,001-1,000,000', '1,000,001-2,000,000']
df['income grouped'] = pd.cut(df['Total income in the last financial year'], bins=f_bins, labels=f_labels, include_lowest=True)
financeBins = df['income grouped'].value_counts(sort=False).reset_index().rename(columns={"index": "income", 0: "count"})


In [ ]:
# For data tables

s_agg_cols = ['Total number of staff',
 'Total number of qualified archivists',
 'Total number of non-qualified archivists',
 'Total number of qualified curators',
 'Total number of records management staff',
 'Total number of conservation staff',
 'Total number of education and learning officers',
 'Total number of project staff',
 'Total number of onsite volunteers',
 'Total number of online volunteers',
 'Total number of volunteer hours',]

f_agg_cols = [ 'Total revenue expenditure in the last financial year',
 'Total capital expenditure in the last financial year',
 'Total income in the last financial year',]

h_agg_cols= ['Cubic metres of analogue archive holdings',
 'Percentage of archival holdings stored in environmental conditions that conform to BS4971',
 'Cubic metres of vacant archive storage',
 'Cubic metres of shelf space in out-stores',
 'Cubic metres of local studies material held in archive storage areas',
 'Cubic metres of analogue records management holdings',
 'Gigabytes (GB) of born-digital archival holdings']

access_cols = ['Percentage of collection with a full catalogue available online',
'Percentage of collection fully catalogued (at least mandatory elements of ISAD(g))?',
'Percentage of collection completely unlisted',
'Analogue items produced in the last year',
'Born-digital items accessed in the last year',
'Visitors to your archive service',
'Hours per week is your archive service open',
'Weeks per year is your archive service open to the public',
'Reader spaces available']

outreach_cols= ['Exhibitions/displays held in-house',
 'Exhibitions/displays held at external venues',
 'Exhibitions held online',
 'People in total that viewed your exhibitions/displays',
 'Learning events held in-house',
 'Learning events held at external venues',
 'Learning events held online',
 "People in total that attended your archive service's learning events"]

def stats_table(df, cols):
  stats = df[cols].agg(['count', 'max', 'min', 'sum', 'mean', 'median']).round(2)
  stats = stats.transpose()
  new_col_names = ['responses', 'highest', 'lowest', 'sum', 'average', 'median']
  stats.columns = new_col_names
  stats_df =stats.reset_index(names='')
  return stats_df

vol_cols = ['Total number of onsite volunteers', 'Total number of online volunteers']

# response rates for each question (column)
# Replace empty strings with NaN
df.replace('', np.nan, inplace=True)
# Total number of responses
total_responses = len(df)
# Calculate the count of non-null values (responses) in each column
response_counts = df.count()
# Calculate the response rate for each column, accounting for null values
response_rates = round((response_counts / total_responses) * 100,2).astype(str) + '%'
response_rates = response_rates.to_frame(name='response_rates').reset_index().rename(columns={"index": "Survey Question", 'response_rates': "Response Rate"})
response_rates

orig_data = df.iloc[:, :-9] #discount merged data
total_responses = len(orig_data)
r_counts = orig_data.count()
r_rates = round((r_counts / total_responses) * 100,2) #.astype(str) + '%'
r_rates = r_rates.to_frame(name='response_rates').reset_index().rename(columns={"index": "question", 'response_rates': "response rates"})
rr = r_rates['response rates'].mean().round(2).astype(str) + '%'

def add_space_before_last_three_chars(postal_code):
  if len(postal_code) > 3:

        # Check if the character before the last 3 characters is not a space

        if postal_code[-4] != ' ':

            modified_postcode = postal_code[:-3] + ' ' + postal_code[-3:]

            return modified_postcode

  return postal_code

df['Formatted Postcode'] = df['Postcode'].apply(add_space_before_last_three_chars)
postal_codes = df["Formatted Postcode"].values.astype('str').tolist()

# lookup postcodes
nomi = pgeocode.Nominatim('gb')

df['lat'] = df['Formatted Postcode'].apply(lambda x: nomi.query_postal_code(x).latitude)
df['lon'] = df['Formatted Postcode'].apply(lambda x: nomi.query_postal_code(x).longitude)

gdf_regions = gdf_regions.to_crs("WGS84")
centroid = gdf_regions.geometry.centroid

# Calculate the mean latitude and longitude of the centroids
center_lat = centroid.y.mean()
center_lon = centroid.x.mean()


df.apply(pd.Series.value_counts)


In [ ]:
# Functions to generate graphs:

# Function to calculate percentages by category
def staff_percentages_by_category(df, cat_col, column_name):
    results = []

    # Filter out NaN values
    df_filtered = df[df[column_name].notna()]

    # Group by category
    grouped_data = df_filtered.groupby(cat_col)

    # Iterate over each group
    for category, group in grouped_data:
        total_archives = len(group)

        # Calculate counts for zero and non-zero qualified archivists
        zero_qualified_count = (group[column_name] == 0).sum()
        non_zero_qualified_count = (group[column_name] > 0).sum()

        # Calculate percentages
        zero_qualified_perc = round((zero_qualified_count / total_archives) * 100, 2)
        non_zero_qualified_perc = round((non_zero_qualified_count / total_archives) * 100, 2)

        job_name = column_name.replace('Total number of ', '').strip()

        # Append results to the list
        results.append({
            f'{cat_col}': category,
            job_name: 'None',
            'Count': zero_qualified_count,
            'Percentage': zero_qualified_perc
        })
        results.append({
            f'{cat_col}': category,
            job_name: 'Yes',
            'Count': non_zero_qualified_count,
            'Percentage': non_zero_qualified_perc
        })

    # Create a DataFrame from the results
    result_df = pd.DataFrame(results)
    return result_df

# Function to calculate % archives having zero staff vs some staff
def calc_zero_perc_staff(df, column_name):
    # Filter out NaN values
    filtered_data = df[df[column_name].notna()]

    # Calculate total number of archives after filtering out NaN values
    total_archives = len(filtered_data)

    # Calculate counts for zero and non-zero values
    zero_count = (filtered_data[column_name] == 0).sum()
    non_zero_count = (filtered_data[column_name] > 0).sum()

    # Calculate percentages
    zero_percentage = round((zero_count / total_archives) * 100, 2)
    non_zero_percentage = round((non_zero_count / total_archives) * 100, 2)

    job_name = column_name.replace('Total number of ', '').strip()

    # Create a new DataFrame
    data = {
        job_name: [f'No {job_name}', f'Has {job_name}'],
        'Count': [zero_count, non_zero_count],
        'Percentage': [zero_percentage, non_zero_percentage]
    }
    result_df = pd.DataFrame(data)
    result_df = result_df.sort_values(by='Percentage', ascending=False)
    return result_df

def volunteers_bar_chart(df, group_by_column, target_cols):
    # Select only columns for summing, and include the grouping column

    grouped_df = df.groupby(group_by_column)[target_cols].sum().reset_index()

    # Melt the DataFrame to long format
    melted_df = pd.melt(grouped_df, id_vars=[group_by_column], value_vars=['Total number of onsite volunteers', 'Total number of online volunteers'],
                        var_name='Volunteer Type', value_name='Count')

    # Sort by the count of onsite volunteers
    melted_df['sort_order'] = melted_df.apply(lambda x: x['Count'] if x['Volunteer Type'] == 'Total number of onsite volunteers' else 0, axis=1)
    melted_df = melted_df.sort_values(by=['sort_order', group_by_column], ascending=[False, True])

    return melted_df

def calc_fzero_perc(df, column_name):
    # Filter out NaN values
    filtered_data = df[df[column_name].notna()]

    # Calculate total number of archives after filtering out NaN values
    total_archives = len(filtered_data)

    # Calculate counts for zero and non-zero values
    zero_count = (filtered_data[column_name] == 0).sum()
    non_zero_count = (filtered_data[column_name] > 0).sum()

    # Calculate percentages
    zero_percentage = round((zero_count / total_archives) * 100, 2)
    non_zero_percentage = round((non_zero_count / total_archives) * 100, 2)

    column_name = column_name.replace(" in the last financial year", "").strip().lower()

    # Create a new DataFrame
    data = {
        column_name: [f'Zero {column_name}', f'Some {column_name}'],
        'Count': [zero_count, non_zero_count],
        'Percentage': [zero_percentage, non_zero_percentage]
    }
    result_df = pd.DataFrame(data)
    return result_df

def calc_zero_perc(df, column_name):
    # Filter out NaN values
    filtered_data = df[df[column_name].notna()]

    # Calculate total number of archives after filtering out NaN values
    total_archives = len(filtered_data)

    # Calculate counts for zero and non-zero values
    zero_count = (filtered_data[column_name] == 0).sum()
    non_zero_count = (filtered_data[column_name] > 0).sum()

    # Calculate percentages
    zero_percentage = round((zero_count / total_archives) * 100, 2)
    non_zero_percentage = round((non_zero_count / total_archives) * 100, 2)

    column_name = column_name.lower()

    # Create a new DataFrame
    data = {
        column_name: [f'Zero {column_name}', f'Some {column_name}'],
        'Count': [zero_count, non_zero_count],
        'Percentage': [zero_percentage, non_zero_percentage]
    }
    result_df = pd.DataFrame(data)
    return result_df

def opening_stats(data, cat_col, open_col):
    # Group by category and calculate value counts for the open_column within each group
    grouped = data.groupby(cat_col)[open_col].value_counts(normalize=False).unstack(fill_value=0)

    # Calculate the percentages
    percentages = data.groupby(cat_col)[open_col].value_counts(normalize=True).mul(100).round(1).unstack(fill_value=0)

    # Combine counts and percentages into a single DataFrame
    combined = grouped.add_suffix('_count').join(percentages.add_suffix('_percentage'))

    # Sort by the percentage of 'Yes' responses
    if 'Yes_percentage' in combined.columns:
        combined = combined.sort_values(by='Yes_percentage', ascending=True)

    return combined


def _overall_perc(df, _cols):
    """
    Calculate the overall percentage of each type of exhibition/event

    """
    # Calculate the total number
    total_s = df[_cols].sum()

    # Calculate the overall total number
    overall_total = total_s.sum()

    # Calculate the percentage of each type
    percentage_ = round((total_s / overall_total) * 100,2)

    # Combine the counts and percentages into a DataFrame
    _overall = pd.DataFrame({
        'count': total_s,
        'percentage': percentage_
    }).sort_values(by='count', ascending=False)

    return _overall

# List of exhibition columns
exhibition_cols = ['Exhibitions/displays held in-house',
 'Exhibitions/displays held at external venues',
 'Exhibitions held online']

# List of learning columns
learning_cols = ['Learning events held in-house',
 'Learning events held at external venues',
 'Learning events held online']

def calc_exhibitions_perc(df, category_col, target_cols):
  """
  Calculate the percentage per category for each type

  """
  # Group by the category and sum the  cols
  grouped_exhibitions = df.groupby(category_col)[target_cols].sum()

  # Calculate the total number of exhibitions for each category
  grouped_exhibitions['totals'] = grouped_exhibitions.sum(axis=1)

  # Calculate the percentage of each type
  for col in target_cols:
      grouped_exhibitions[f'percentage_{col}'] = round((grouped_exhibitions[col] / grouped_exhibitions['totals']) * 100,2)

  # Sort the results by the total number of exhibitions
  exhibitions_sort = grouped_exhibitions.sort_values(by='totals', ascending=False)

  return exhibitions_sort

people_cols = ['People in total that viewed your exhibitions/displays', "People in total that attended your archive service's learning events"]

sm_cols = [ #'Not Applicable',
'Bluesky',
'Blog',
'Facebook',
'Flickr',
'Instagram',
'Pinterest',
'Mastodon',
'TikTok',
'Threads',
'Twitter/X',
'YouTube']


In [ ]:
# CMS funtions

others =['MODES',
 'Custom-made Access database',
 'Modes',
 'Modes Complete'
 'Axiell',
 'Koha',
 'ARK by Heliocentrix',
 'AtoM',
 'INVU',
 'Adlib',
 'HDMS',
 'Bespoke',
 'Preservica',
 'SDS Archive Virtual',
 'Textworks',
 'RAFTS',
 'inhouse system',
 'Mimsy',
 'MS Access',
 'Spectrum',
 'Cardbox',
 'MIMSY',
 'MIMSY (Axiell)',
 'Archives Hub',
 'OCLC',
 'Manage your collections (National Archives)',
 'Atoll from Ammba (based on CORALS)',
 'Sirsi',
 'Archive Index+ (System Simulation)',
 'BFI National Film Archive',
 'Equella',
 'MIMSY XG ',
 'Arkivum',
 'Talis Alto; iBase',
 'Index+ (for Local Studies catalogue) ',
 'Axiell Collections',
 'TALIS',
 'Syrsi Dynix - Workflows',
 'PRM (FileTrail) for Records Management',
 'Seax']

def add_yes_no_col(df, col_name, systems_list):
    # Normalize the text in the specified column and list

    system_list = [ele for ele in systems_list if ele.strip()]
    systems_list = [x.lower() for x in systems_list]
    df[col_name] = df[col_name].str.strip().str.lower()

    # Function to check if any keyword is present in the text
    def check_systems(text, systems):
        return any(system in text for system in systems)

    # Create a new column with Yes/No values
    new_col_name = col_name.replace('(please specify):', '').strip()
    df[new_col_name] = df[col_name].apply(lambda x: 'Yes' if check_systems(x, systems_list) else 'No')

    return df

df = add_yes_no_col(df, 'Other CMS (please specify):', others)

# function to count 'yes' columns and get percentages

def col_counts_percentages(df, cols):
    # Apply value counts and filter for 'Yes'
    counts = df[cols].apply(pd.Series.value_counts).loc['Yes']

    # Drop NaN values
    counts.dropna(inplace=True)

    # Sort values
    sorted_counts = counts.sort_values(ascending=False)
    sorted_counts = sorted_counts.rename_axis('name').to_frame('count')

    # Calculate percentages
    #total = counts.sum()
    total = len(df)
    percentages = (counts / total) * 100
    sorted_percentages = percentages.sort_values(ascending=False)
    sorted_percentages = sorted_percentages.rename_axis('name').to_frame('percentage')

    # Merge counts and percentages
    result = pd.merge(sorted_counts, sorted_percentages, on='name')

    return result

system_columns = [ 'No CMS',
 'AdLib',
 'ArchivEra (Lucidea)',
 'ArchivesSpace',
 'AtoM',
 'Axiell Collections',
 'Axiell DAMS',
 'Axiell Emu',
 'CALM',
 'CollectiveAccess',
 'CollectionsBase (Orangeleaf Systems)',
 'Collections Index (System Simulation)',
 'ContentDM (OCLC)',
 'CuadraStar SKCA (Lucidea)',
 'Dryad (Max Communications)',
 'eHive (Vernon Systems)',
 'Eloquent (Lucidea)',
 'EmbARK (Gallery Systems)',
 'Epexio (Metadatis)',
 'Extensis Portfolio',
 'Hyku (Hydra-In-A-Box)',
 'Islandora',
 'Keep Thinking',
 'M2A (Minisis)',
 'Omeka Classic',
 'Omeka.net',
 'OmekaS',
 'PastPerfect',
 'PastView (TownsWeb Archiving Ltd)',
 'Picturae - Memorix Archives',
 'Picturae - Memorix Maior',
 'Proficio (Re:Discovery)',
 'Qi',
 'scopeArchiv',
 'SDS Archive Manager (SDS Group)',
 'Soutron Archive',
 'Spydus (Civica)',
 'TMS Media Studio (Gallery Systems)',
 'VernonCMS',
 'Widen',
 'XTF',
'Other CMS'
]

systems = col_counts_percentages(df, system_columns)

# Function to get percentage usage and compare usage of any system vs no system overall

def calculate_system_usage(data, system_columns, none_column):
    """
    Calculate and compare the usage of systems vs. no system.

    Parameters:
    - data (pd.DataFrame): The DataFrame containing the survey data.
    - system_columns (list of str): List of columns representing the different systems.
    - none_column (str): The column representing the 'None' option.

    Returns:
    - pd.DataFrame: DataFrame with counts and percentages of system usage.
    """
    # Calculate the count and percentage of usage for each system
    system_usage = data[system_columns].apply(pd.Series.value_counts).loc['Yes']

    # Drop NaN values
    system_usage.dropna(inplace=True)

    system_usage = system_usage.sort_values(ascending=False)
    total_system_usage = system_usage.sum()
    #total_system_usage = len(data)
    system_percentage = round((system_usage / total_system_usage) * 100, 2)

    # Calculate the count and percentage of respondents using at least one system vs. none
    total_respondents = system_usage.sum()
    none_count = data[none_column].value_counts().get('Yes', 0)
    with_system_count = total_respondents - none_count

   # none_percentage = round((none_count / total_respondents) * 100,2)
   # with_system_percentage = round((with_system_count / total_respondents) * 100,2)

    total_s = len(data)
    none_percentage = round((none_count / total_s) * 100,2)
    with_system_percentage = round((with_system_count / total_s) * 100,2)

    # Create a DataFrame for the overall usage summary
    summary_data = {
        'Usage': ['Yes', 'No'],
        'Count': [with_system_count, none_count],
        'Percentage': [with_system_percentage, none_percentage]
    }
    summary_df = pd.DataFrame(summary_data)

    # Create a DataFrame for the individual system usage
    individual_system_usage_df = pd.DataFrame({
        'System': system_usage.index,
        'Count': system_usage.values,
        'Percentage': system_percentage.values
    })

    return summary_df, individual_system_usage_df

#summary_df, individual_system_usage_df = calculate_system_usage(df, system_columns, 'No CMS')

def apply_sys(df, cols):
  for col in cols:
    df[col] = df[col].apply(lambda x: 1 if x == 'Yes' else 0)
  return df

df_sys = df.copy()
CMS_count = apply_sys(df_sys, system_columns)

def system_usage_perc(df, category_col, sys_cols, none_col):
    # Calculate total responses per category - could tick more than one
    df['total'] = df[sys_cols].sum(axis=1)
    responses = df.groupby(category_col)['total'].sum()
    total_per_cat= df.groupby(category_col).size()

    # Calculate sum of system usage per category
    system_sums = df.groupby(category_col)[[col for col in sys_cols]].sum()

    # Calculate number of rows with any system used per category
    system_sums['Any system count'] = system_sums[[col for col in sys_cols if col != none_col]].sum(axis=1)

    # Calculate number of rows with no system used per category
    #system_sums['None'] = system_sums[none_col]
    system_sums['No system count'] = system_sums[none_col]
    #system_sums['CALM count'] = system_sums['CALM']

    # Calculate the percentages
    system_sums['Any system %'] = ((system_sums['Any system count'] / responses) * 100).round(2)
    system_sums['No system %'] = ((system_sums['No system count'] / responses) * 100).round(2)
    #system_sums['CALM %'] = ((system_sums['CALM count'] / total_per_cat) * 100).round(2)

    return system_sums[[#'CALM count', 'CALM %',
                        'Any system count', 'Any system %', 'No system count','No system %']]



other_sys = [
 'Azure cloud storage',
 'Digisafe', 'LibNova'
 'network backup',
 'Manual back-up onto an external hard drive.',
 'NAS',
 'In-house system',
 'Scanning on photocopier and storing in Google Drive',
 'INVU (Virtual Cabinet)',
 'Custom designed',
 'In-house',
 "Oxford University 'Digisafe' digital preservation service ",
 'BFI Film Archive uses',
 'Cyber Duck encrypted pipeline to cloud storage',
 'SCAT tool (SCAT is Curation And Trust)',
 'Preservica',
 'in-house bespoke digital preservation systems', 'NDSA Levels of Digital Preservation',
 'TiTAN TDR (MINISIS Inc)',
 'HPE Records Manager',
 'In house',
 ]
df = add_yes_no_col(df, 'Other system (please specify):', other_sys)

digi_pres_sys_cols = [ 'No digital preservation system',
 'In-house workflows connecting tools into a digital preservation architecture',
 'Archivematica',
 'ArchivesDirect',
 'Arkivum Perpetua',
 'DAITSS',
 'DSpace Direct',
 'EPrints',
 'Fedora',
 'iRODS',
 'Libsafe [Libnova]',
 'Portico',
 'Preservica',
 'Roda',
 'Rosetta - Ex Libris',
 'Samvera',
 'scopeOAIS [scopeArchiv]',
 'Other system',]
digi_pres_overall = col_counts_percentages(df, digi_pres_sys_cols)

df_sys = df.copy()
DPS_count = apply_sys(df_sys, digi_pres_sys_cols)


df['Other(s) (please specify):'] = df['Other(s) (please specify):'].astype(str)
digi_standards = ['dpcmm',
 'spectrum',
 'dcp handbook',
 'museum of london archaeological service storage requirements',
 "oxford university 'digisafe' digital preservation service",
 'ica; open preservation coalition',
 'dcc digital curation lifecycle model',
 'in house system',
 'external company to archival standards.',
 'iso 9001:2025',
 'iso 14721:2012',
 'open archival information system (oais) – reference model','iso 16363: 2012',
 'iso 17068:2017',
 'iso/tr 18492: 2005',
 'iso/iec 19757-2:2008', 'premis',
 'iso/iec 11578:1996',
 'dlf levels of born-digital access',
 'in-house bespoke digital preservation systems', 'ndsa levels of digital preservation',
 'library of congress premis standard']

df = add_yes_no_col(df, 'Other(s) (please specify):', digi_standards)

digi_pres_tools_cols = ['No standard/guide used',
'CoreTrustSeal Trustworthy Digital Repositories Requirements',
'DiAGRAM (Digital Archiving Graphical Risk Assessment Model)',
'Digital Preservation Coalition Rapid Assessment Model (DPC RAM)',
'ISO 16363 (Space data and information transfer systems: Audit and certification of trustworthy digital repositories)',
'NDSA Levels of Digital Preservation',
'OAIS Reference Model',
'Other(s)'
]

digi_pres_tools = col_counts_percentages(allData, digi_pres_tools_cols)

df_sys = allData.copy()
standards_count = apply_sys(df_sys, digi_pres_tools_cols)



In [ ]:
CMS_usage_per_cat = system_usage_perc(CMS_count, "Category", system_columns, 'No CMS')
CMS_usage_per_cat.columns = CMS_usage_per_cat.columns.str.replace("system", "CMS")
CMS_usage_per_cat = CMS_usage_per_cat.sort_values(by='Category', ascending=True)
CMS_usage_per_cat = CMS_usage_per_cat.reset_index()
melted_cms = CMS_usage_per_cat.melt(id_vars=['Category'], value_vars=['Any CMS %', 'No CMS %'], var_name='CMS', value_name='CMS Percentage')

standards_usage_per_cat = system_usage_perc(standards_count, "Category", digi_pres_tools_cols, 'No standard/guide used')
standards_usage_per_cat = standards_usage_per_cat.sort_values(by='Category', ascending=True)
standards_usage_per_cat.columns = standards_usage_per_cat.columns.str.replace("system", "standard")
standards_usage_per_cat = standards_usage_per_cat.reset_index()
melted_standards = standards_usage_per_cat.melt(id_vars=['Category'], value_vars=['Any standard %', 'No standard %'], var_name='Digital Preservation Standard', value_name='Standard Percentage')

DPS_usage_per_cat = system_usage_perc(DPS_count, 'Category', digi_pres_sys_cols, 'No digital preservation system')
DPS_usage_per_cat = DPS_usage_per_cat.sort_values(by='Category', ascending=True)
DPS_usage_per_cat = DPS_usage_per_cat.reset_index()
melted_dms = DPS_usage_per_cat.melt(id_vars=['Category'], value_vars=['Any system %', 'No system %'], var_name='Digital Preservation System', value_name='System Percentage')

melted = pd.merge(melted_cms, melted_dms,  right_index=True, left_index=True)

melted_sys = pd.merge(melted, melted_standards,  right_index=True, left_index=True).drop(['Category_x', 'Category_y'], axis=1)
melted_sys

In [ ]:

'''
the app with more tabs
'''
config = {
    'scrollZoom': False,
    'displayModeBar': False
    }



app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

tabs_styles = {
    'height': '54px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF',
    'color': 'white',
    'padding': '6px'
}

tab_selected_style0 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#e1b941',
    'color': 'white',
    'padding': '6px'
}


tab_selected_style1 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#007dd5',
    'color': 'white',
    'padding': '6px'
}

tab_selected_style2 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#3d96ab',
    'color': 'white',
    'padding': '6px'
}

tab_selected_style3 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#c9334e',
    'color': 'white',
    'padding': '6px'
}

tab_selected_style4 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#af4eb2',
    'color': 'white',
    'padding': '6px'
}

tab_selected_style5 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#479c28',
    'color': 'white',
    'padding': '6px'
}



app.layout = html.Div([
    dcc.Tabs(id="tabs-styled-with-inline", value='tab-1', children=[
        dcc.Tab(label='Overview', value='tab-1', style=tab_style, selected_style=tab_selected_style0),
        dcc.Tab(label='Workforce', value='tab-2', style=tab_style, selected_style=tab_selected_style1),
        dcc.Tab(label='Holdings', value='tab-3', style=tab_style, selected_style=tab_selected_style2),
        dcc.Tab(label='Public Access', value='tab-4', style=tab_style, selected_style=tab_selected_style3),
        dcc.Tab(label='Outreach', value='tab-5', style=tab_style, selected_style=tab_selected_style4),
        dcc.Tab(label='Finance', value='tab-6', style=tab_style, selected_style=tab_selected_style5),
    ], style=tabs_styles),

    #empty graphs to avoid id error
    html.Div(id='tabs-content-inline', children=[
        dcc.Graph(id='cat-graph'),
        dcc.Graph(id='category-graph'),
        dcc.Graph(id='overall-graph'),
        dcc.Graph(id='bin-graph'),
        dcc.Graph(id='volunteer-graph'),
        dcc.Graph(id='map-graph'),
        dcc.Graph(id='fin-graph'),
        dcc.Graph(id='fin-category-graph'),
        dcc.Graph(id='zero-fin-graph'),
        dcc.Graph(id='hold-graph'),
        dcc.Graph(id='all-sys-graph'),
        dcc.Graph(id='sys-graph'),
        dcc.Graph(id='cms-graph'),
        dcc.Graph(id='com-graph'),
        dcc.Graph(id='stor-graph'),
        dcc.Graph(id='cat-online-graph'),
        dcc.Graph(id='cat-unlist-graph'),
        dcc.Graph(id='cat-vs-graph'),
        dcc.Graph(id='opening-stats-graph'),
        dcc.Graph(id='vis-graph'),
        dcc.Graph(id='ex-graph'),
        dcc.Graph(id='all-ex-graph'),
        dcc.Graph(id='soc-graph'),
        dcc.Graph(id='maps-graph'),
        dcc.RadioItems(id='category-selector'),
        dcc.Dropdown(id='col-selector'),
        dcc.Dropdown(id='f-selector'),
        dcc.Dropdown(id='sys-selector'),
        dcc.Dropdown(id='ex-learn-selector')
    ])
] )

@callback(Output('tabs-content-inline', 'children'),
            Input('tabs-styled-with-inline', 'value')
          )

def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.Div(children='Survey Overview', style={'textAlign': 'center', 'color': '#b9921d', 'fontSize': 30, 'margin': '20px 0'}),

            html.Div(children=[
        html.P([
            "This dashboard visualises responses from a survey carried out by The National Archives between Jan and Mar 2024. ",
            html.Br(),
            "The survey aimed to gain better insight into the health of the achive sector, allow benchmarking and aid advocacy. ",
            html.Br(),
            "The results are split into 5 sections: workforce, holdings, access, outreach and finance. ",
            html.Br(),
            "This tab provides an overview of the survey responses by category of archive and by region. ",
            html.Br(),
            "Nearly a quarter of responses came from London archives, and the largest category to respond was Local Authority archives. "

        ], style={'textAlign': 'center', 'fontSize': 18, 'margin': '20px', 'padding':'20px'})
    ]),


    # html.Span(children=['This dashboard visualises responses from a survey carried out by The National Archives between Jan and Mar 2024.'],
    #           style={"border":"2px yellow solid", 'margin': '30px auto', 'padding':'30px','padding-left':'30px','textAlign': 'center' }),

      html.Div(
      [
 dbc.Row(
                [
                    dbc.Col(
                        dbc.Card(
                            [
                                dbc.CardBody("Total Number of Responses"),
                                dbc.CardBody(f"{len(df)}", style={'font-size': '20px'})
                            ],
                            className="mb-3",
                            style={'backgroundColor': '#e9cd76', 'height':'100%', 'textAlign': 'center'}
                        ),
                        width=4,
                        align="center",

                    ),
                    dbc.Col(
                        dbc.Card(
                            [
                                dbc.CardBody("Average Response Rate"),
                                dbc.CardBody(rr, style={'font-size': '20px'})
                            ],
                            className="mb-3",
                            style={'backgroundColor': '#e9d776', 'height':'100%', 'textAlign': 'center'}
                        ),
                        width=4,
                        align="center",
                    )
                ],
                style={"margin": "10 auto"}, justify='center',
            ),


]),

    html.Div(children=[
        dcc.RadioItems(options=['Category', 'Postcode Region'],
                       value='Category', inline=True,
                       id='category-selector', style={'margin': '30px auto'})
    ], style={'textAlign': 'left', 'padding': '10px'}),

    html.Div([
      html.Div([
            dcc.Graph(id='cat-graph', style={'height': '650px', 'margin': '10px auto'})
        ], style={'width': '45%', 'display': 'inline-block', 'verticalAlign': 'top'}),

      html.Div([
            dcc.Graph(id='maps-graph', style={'height': '650px', 'margin': '10px auto'})
        ], style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'textAlign': 'right'}),

      html.Div([
    html.Div(children=[
        dash_table.DataTable(data=response_rates.to_dict('records'), page_size=5,fixed_rows={'headers': True}, style_table={'overflowX': 'auto', 'width': '90%', 'margin-top': '40px', 'margin-left':'20px'},
        style_cell={'textAlign': 'center', 'padding-left': '30px'})
    ])

    ])
        ])

    elif tab == 'tab-2':
        return html.Div([

            html.Div(children='Staff Overview', style={'textAlign': 'center', 'color': '	royalblue', 'fontSize': 30, 'margin': '20px 0'}),

    html.Div(children=[
        dash_table.DataTable(data=stats_table(df, s_agg_cols).to_dict('records'), page_size=5, style_table={'overflowX': 'auto', 'width': '70%', 'margin': '0 auto'},
        style_cell={'textAlign': 'center'}
        )
    ]),

    html.Div([
      html.Div([
            dcc.Graph(id='bin-graph', style={'height': '600px', 'margin': '10px auto'})
        ], style={'width': '80%'}
    )], style={'textAlign': 'centre'}),

    html.Div(children=[
        dcc.RadioItems(options=['Category', 'Postcode Region'],
                       value='Category', inline=True,
                       id='category-selector', style={'margin': '20px auto'})
    ], style={'textAlign': 'left', 'padding-left': '20px'}),

    html.Div([
        dcc.Dropdown(
            id='col-selector',
            options=[{'label': col, 'value': col} for col in df.columns if col.startswith('Total number of ')],
            value=[col for col in df.columns if col.startswith('Total number of ')][1],
            style={'margin': '20px auto', 'padding-left': '20px'}
        )
    ], style={'textAlign': 'left', 'width': '40%'}),

    html.Div([
        html.Div([
            dcc.Graph(id='category-graph', style={'height': '600px', 'margin': '30px auto'})
        ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top'}),

        html.Div([
            dcc.Graph(id='overall-graph', style={'height': '650px', 'width':'1000', 'margin': '30px auto'})
        ], style={'width': '68%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'textAlign': 'right'}),

        html.Div([
        dcc.Graph(id='volunteer-graph', style={'height': '600px'})
    ], style={'textAlign': 'center', 'padding': 10, 'flex':1})
        ])

    elif tab == 'tab-3':
        return html.Div([
            html.Div(children='Holdings Overview', style={'textAlign': 'center', 'color': 'teal', 'fontSize': 30, 'margin': '20px 0'}),

    html.Div(children=[
        dash_table.DataTable(data=stats_table(df, h_agg_cols).to_dict('records'), page_size=5, style_table={'overflowX': 'auto', 'width': '85%', 'margin': '0 auto'},
        style_cell={'textAlign': 'center'}
        )
    ]),

    html.Div(children=[
        dcc.RadioItems(options=['Category', 'Postcode Region'],
                       value='Category', inline=True,
                       id='category-selector', style={'margin': '20px 0'})
    ], style={'textAlign': 'left', 'padding-left': '20px'}),


    html.Div([
            dcc.Graph(id='hold-graph', style={'height': '800px', 'margin': '30px auto'})
        ], style={'width': '80%', 'padding-left': '20px'}),

    html.Div([
    html.Div([
            dcc.Graph(id='stor-graph', style={'height': '650px', 'width':'1000px', 'margin': '20px auto'})
        ], style={'width': '67%', 'display': 'inline-block', 'verticalAlign': 'top'}),

    html.Div([
            dcc.Graph(id='com-graph', style={'height': '600px', 'margin': '30px auto'})
        ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'textAlign': 'right'}),


    html.Div([
      dcc.Dropdown(
            id='sys-selector',
            options=[
            {'label': 'CMS', 'value': 'CMS'},
            {'label': 'Digital Preservation System', 'value': 'Digital Preservation System'},
            {'label': 'Digital Preservation Standard', 'value': 'Digital Preservation Standard'}
        ],
            value='CMS',
            style={'margin': '20px 0'}
        )
    ], style={'textAlign': 'left', 'width': '40%', 'padding-left': '20px'}),

    html.Div([
    html.Div([
            dcc.Graph(id='all-sys-graph', style={'height': '650px', 'margin': '30px auto'})
        ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top'}),

    html.Div([
            dcc.Graph(id='sys-graph', style={'height': '650px', 'width':'1000', 'margin': '30px auto'})
        ], style={'width': '67%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'textAlign': 'right'}),

    html.Div([
        dcc.Graph(id='cms-graph', style={'height': '500px'})
    ], style={'textAlign': 'center', 'padding': 10, 'flex':1})

        ])

    elif tab == 'tab-4':
        return html.Div([
            html.Div(children='Public Access Overview', style={'textAlign': 'center', 'color': 'crimson', 'fontSize': 30, 'margin': '20px 0'}),

        html.Div(children=[
        dash_table.DataTable(data=stats_table(df, access_cols).to_dict('records'), page_size=5, style_table={'overflowX': 'auto', 'width': '85%', 'margin': '0 auto'},
        style_cell={'textAlign': 'center'})
        ]),

        html.Div([
        html.Div([
            dcc.Graph(id='cat-online-graph', style={'height': '500px', 'margin': '20px auto'})
        ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),

        html.Div([
            dcc.Graph(id='cat-unlist-graph', style={'height': '500px', 'margin': '20px auto'})
        ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'})
        ], style={'textAlign': 'left', 'padding-left': '20px'}),

        html.Div(children=[
        dcc.RadioItems(options=['Category', 'Postcode Region'],
                       value='Category', inline=True,
                       id='category-selector', style={'margin': '20px 0'})
        ], style={'textAlign': 'left', 'padding-left': '20px'}),

      html.Div([
      html.Div([
            dcc.Graph(id='cat-vs-graph', style={'height': '600px', 'margin': '10px auto'})
        ], style={'width': '85%'}
    )], style={'textAlign': 'centre', 'padding-left': '20px'}),

      html.Div([
        dcc.Graph(id='opening-stats-graph', style={'height': '800px', 'width':'1400px', 'margin-bottom': '10px'})
    ], style={'textAlign': 'center', 'padding': 10, 'flex':1}),

      html.Div([
        dcc.Graph(id='vis-graph', style={'height': '600px', 'margin-top': '10px'})
    ], style={'textAlign': 'center', 'padding': 10, 'flex':1})

        ])

    elif tab == 'tab-5':
        return html.Div([
            html.Div(children='Outreach Overview', style={'textAlign': 'center', 'color': 'darkmagenta', 'fontSize': 30, 'margin': '20px 0'}),

            html.Div(children=[
        dash_table.DataTable(data=stats_table(df, outreach_cols).to_dict('records'), page_size=5, style_table={'overflowX': 'auto', 'width': '80%', 'margin': '0 auto'},
        style_cell={'textAlign': 'center'}
        )
    ]),
            html.Div([
        dcc.Dropdown(
            id='ex-learn-selector',
            options=[
                {'label': 'Exhibitions', 'value': 'Exhibitions'},
                {'label': 'Learning Events', 'value': 'Learning Events'}
            ],
            value='Exhibitions',
            style={'margin': '20px 0'}
        ),
    ], style={'width': '40%', 'display': 'inline-block', 'padding-left': '20px'}),

            html.Div([
        dcc.Graph(id='all-ex-graph', style={'height': '500px', 'margin': '20px auto', 'padding-left': '30px'})
    ]),

            html.Div(children=[
        dcc.RadioItems(options=['Category', 'Postcode Region'],
                       value='Category', inline=True,
                       id='category-selector', style={'margin': '20px 0'})
        ], style={'textAlign': 'left', 'padding-left': '20px'}),

    html.Div([
        dcc.Graph(id='ex-graph', style={'height': '800px', 'margin': '20px auto', 'padding-left': '30px'})
    ]),

    html.Div([
        dcc.Graph(id='soc-graph', style={'height': '800px', 'width':'1200px', 'margin-bottom': '10px'})
    ], style={'textAlign': 'left', 'padding': 10, 'flex':1})

            ])

    elif tab == 'tab-6':
        return html.Div([
            html.Div(children='Finance Overview', style={'textAlign': 'center', 'color': 'green', 'fontSize': 30, 'margin': '20px 0'}),

    html.Div(children=[
        dash_table.DataTable(data=stats_table(df, f_agg_cols).to_dict('records'), page_size=5, style_table={'overflowX': 'auto', 'width': '75%', 'margin': '0 auto'},
        style_cell={'textAlign': 'center'}
        )
    ]),

    html.Div([
      html.Div([
            dcc.Graph(id='fin-graph', style={'height': '600px', 'margin': '10px auto'})
        ], style={'width': '80%'}
    )], style={'textAlign': 'centre', 'padding-left': '20px'}),

    html.Div([
        dcc.Dropdown(
            id='f-selector',
            options=[{'label': col, 'value': col} for col in df.columns if col.endswith('financial year')],
            value=[col for col in df.columns if col.endswith('financial year')][2],
            style={'margin': '20px 0', 'padding-left': '20px'}
        )
    ], style={'textAlign': 'left', 'width': '40%',}),

      html.Div([
        html.Div([
        dcc.Graph(id='map-graph', style={'height': '600px'})
    ], style={'textAlign': 'center', 'padding': 10, 'flex':1}),

        html.Div([
        dcc.Graph(id='fin-category-graph', style={'height': '600px', 'margin': '10px auto'})
        ], style={'width': '35%', 'display': 'inline-block', 'verticalAlign': 'top'}),

        html.Div([
        dcc.Graph(id='zero-fin-graph', style={'height': '600px', 'margin': '10px auto'})
        ], style={'width': '60%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'textAlign': 'right'})

        ])

# Callback to update the graphs
@callback(
    [Output('cat-graph', 'figure'),
     Output('maps-graph','figure')],
    Input('category-selector', 'value'))

def update_overview_graphs(selected_category):
  custom_color_sequence1 = [
      "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd",
      "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf",
      "#aec7e8", "#ffbb78", "#98df8a", "#ff9896", "#c5b0d5",
      "#ebd183", "#f7b6d2", "#c7c7c7"
  ]
  custom_color_sequence2 = [
      '#e0b941','#e8cb72','#e3c157','#e7c96d','#ebd183', "#eed999",'#eacf7e']

  center_lat = df['lat'].mean()
  center_lon = df['lon'].mean()

  if selected_category == 'Category':
      maps_fig = px.scatter_mapbox(df, lat='lat', lon='lon', color='Category',
                              color_discrete_sequence=custom_color_sequence1,
                              opacity=0.5,
                              center={"lat": center_lat, "lon": center_lon}, zoom=5)
  else:
      maps_fig = px.scatter_mapbox(df, lat='lat', lon='lon', color='Postcode Region',
                              color_discrete_sequence=custom_color_sequence2,
                              opacity=0.5,
                              center={"lat": center_lat, "lon": center_lon}, zoom=6)
      maps_fig.update_layout(showlegend=False)

  maps_fig.update_layout(
      mapbox=dict(
          style="carto-positron",
          zoom=5,  # Initial zoom level
          center=dict(lat=center_lat, lon=center_lon),
      ), height=550,
      margin=dict(b=60, t=30, l=10, r=10),
      uirevision='locked'  # Lock user interface interactions
  )
  maps_fig.update_traces(marker={"size": 20})

  categories_count = df[selected_category].value_counts().reset_index().rename(columns={"index": "value", 'count': "responses"})
  categories_count = categories_count.sort_values(by='responses', ascending=True)
  cat_fig = px.bar(categories_count, y=selected_category, x='responses', orientation='h',
             labels={selected_category: selected_category, 'responses': 'responses'},
             title=f'Responses per {selected_category}')
  cat_fig.update_traces(marker_color='rgb(237, 173, 8)')
  cat_fig.update_layout(xaxis_title='Number of Responses', yaxis_title='Archive Category',
                  template='plotly_white', height=550, margin=dict(b=60, t=30, l=10, r=10))

  return cat_fig, maps_fig

@callback(
    [Output('hold-graph', 'figure'),
    Output('stor-graph', 'figure'),
    Output('com-graph', 'figure'),
    Output('all-sys-graph', 'figure'),
    Output('sys-graph', 'figure'),
    Output('cms-graph', 'figure')],
    [Input('category-selector', 'value'),
    Input('sys-selector', 'value')]
    )

def update_holdings_graphs(selected_category, selected_system):
  holdings = allData.groupby(selected_category)[['Cubic metres of analogue archive holdings', 'Gigabytes (GB) of born-digital archival holdings']].sum()
  holdings = holdings.sort_values(by='Cubic metres of analogue archive holdings', ascending=True)

# Create figure
  hold_fig = go.Figure()

# Add Cubic Metres bar trace
  hold_fig.add_trace(go.Bar(
    y=holdings.index,
    x=holdings['Cubic metres of analogue archive holdings'],
    name='Cubic Metres',
    marker=dict(color='rgb(118,78,159)'),
    orientation='h',
    xaxis='x1',
    yaxis='y1',
    offsetgroup=1
  ))

# Add Gigabytes bar trace
  hold_fig.add_trace(go.Bar(
    y=holdings.index,
    x=holdings['Gigabytes (GB) of born-digital archival holdings'],
    name='Gigabytes',
    marker=dict(color='#3dada5'),
    orientation='h',
    xaxis='x2',
    yaxis='y1',
    offsetgroup=2
  ))

# Update layout for dual x-axes
  hold_fig.update_layout(
    title=f'Comparison of Cubic Metres and Gigabytes by {selected_category}',
    yaxis=dict(
        title=f'Archive {selected_category}',
    ),

    xaxis=dict(
        title='Cubic Metres',
        titlefont=dict(
            color='rgb(118,78,159)'
        ),
        tickfont=dict(
            color='rgb(118,78,159)'
        ),
        side='bottom',
    ),
    xaxis2=dict(
        title='Gigabytes',
        titlefont=dict(
            color='#3dada5'
        ),
        tickfont=dict(
            color='#3dada5'
        ),
       overlaying='x',
       side='top'
        #type='log'
    ),
    legend=dict(
        x=1.0,
        y=0.1,
        orientation='v'
    ),
    margin=dict(
        t=120  # Add space at the top for the title
    ),
    barmode='group',
    plot_bgcolor='white',
  )
  hold_fig.update_layout(
    width=1200,  # Set width of the plot
    height=800,  # Set height of the plot
  )

  df['Commercial storage for archival holdings'].mask(df['Commercial storage for archival holdings'] == 'Na fyddwn / No', 'No', inplace=True)
  counts = df['Commercial storage for archival holdings'].value_counts()
  percs = df['Commercial storage for archival holdings'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
  com_stor = pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])
  com_stor = com_stor.reset_index(names='Commercial storage for archival holdings')

  com_fig = px.pie(
  com_stor,
  names='Commercial storage for archival holdings',
  values='count',
  color='Commercial storage for archival holdings',
  color_discrete_map={
  'Yes': '#58b7c4',
  'No': '#cb6b90'
  },
  title='Percentage of Archives with<br>Commercial Storage',
  hole=0.3  # This creates a donut chart
  )

  # Update the layout for the pie chart
  com_fig.update_traces(textposition='inside', textinfo='percent')
  com_fig.update_layout(
  uniformtext_minsize=10,
  uniformtext_mode='hide',
  template='plotly_white',
  height=450,
  #width=400,
  margin=dict(l=20, r=10, t=160, b=20),  # Adjust margins to align chart
  legend=dict(
      orientation="h",  # Horizontal orientation
      yanchor="top",  # Position legend above the chart
      y=1.15,
      xanchor="center",
      x=0.5
  )
  )
  com_fig.update_layout(legend_traceorder="reversed")
  com_fig.update_traces(
  hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent:.1%}'
  )

  cstor = allData.groupby(selected_category)['Commercial storage for archival holdings'].value_counts(normalize=True).mul(100).round(1)
  cstor = cstor.to_frame().reset_index().rename(columns={'proportion':'percentage'})
  cstor = cstor.sort_values(['Commercial storage for archival holdings','percentage'], ascending=[False, False])
  stor_fig = px.bar(cstor,
                          y=selected_category,
                          x='percentage',
                          color='Commercial storage for archival holdings',
                          color_discrete_map={
                              'Yes': '#58b7c4',
                              'No': '#cb6b90'},
                          text='percentage',
                          barmode='stack',
                          orientation='h',
                          height=650,width=950,
                          labels={selected_category: selected_category, 'Commercial storage for archival holdings': 'Commercial storage'},
                          title=f'Commercial Storage for Archival Holdings per {selected_category}')

  stor_fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')
  stor_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                              xaxis_title='Percentage', yaxis_title=f'Archive {selected_category}',
                              template='plotly_white', margin=dict(l=20, r=10, t=120, b=20),
                              legend=dict(
          orientation="h",  # Horizontal orientation
          yanchor="top",  # Position legend above the chart
          y=1.1,
          xanchor="center",
          x=0.5)
  )

  if selected_system == 'CMS':
    summary_df, individual_system_usage_df = calculate_system_usage(df, system_columns, 'No CMS')
    all_sys_fig = px.bar(summary_df, x='Usage', y='Count', text='Percentage',
              labels={'Usage': 'CMS', 'Count': 'Count'},
              title='Percentage of Respondents<br>Using a CMS',
              color='Usage',
              color_discrete_map={'Yes':'#3d96ac',
                                  'No':'#ac3d5f'})
    all_sys_fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    all_sys_fig.layout.update(uniformtext_minsize=10, uniformtext_mode='hide',
                    xaxis_title='System Usage', yaxis_title='Number of Respondants',
                    template='plotly_white')

    CMS_usage_per_cat = system_usage_perc(CMS_count, selected_category, system_columns, 'No CMS')
    CMS_usage_per_cat.columns = CMS_usage_per_cat.columns.str.replace("system", "CMS")
    CMS_usage_per_cat = CMS_usage_per_cat.reset_index()

    CMS_usage_per_cat = CMS_usage_per_cat.sort_values(by='Any CMS %', ascending=True)
    melted_cms = CMS_usage_per_cat.melt(id_vars=[selected_category], value_vars=['Any CMS %', 'No CMS %'], var_name='CMS', value_name='CMS Percentage')

    sys_fig = px.bar(melted_cms,
                            y=selected_category,
                            x='CMS Percentage',
                            color='CMS',
                            color_discrete_map={
                                'Any CMS %': '#3d96ac',
                                'No CMS %': '#ac3d5f'},
                            text='CMS Percentage',
                            barmode='stack',
                            orientation='h',
                            height=650, width=1000,
                            labels={selected_category: selected_category, 'CMS': 'CMS'},
                            title=f'CMS Usage per {selected_category}')

    sys_fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')
    sys_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                                xaxis_title='Percentage', yaxis_title=f'Archive {selected_category}',
                                template='plotly_white')

    systems_g = systems.sort_values(by='percentage', ascending=True)

    cms_fig = px.bar(systems_g, y=systems_g.index, x='percentage', title='Percentage of Respondents Using Each System', height=700,width=1200,
                labels={'percentage': 'Percentage', 'name': 'CMS'}, orientation='h', template='plotly_white' )
    cms_fig.update_traces(marker_color='#45a6bd')
    cms_fig.layout.update(uniformtext_minsize=10, uniformtext_mode='hide',
                xaxis_title='Percentage', yaxis_title='System name',
                template='plotly_white')

    return hold_fig, stor_fig, com_fig, all_sys_fig, sys_fig, cms_fig

  elif selected_system == 'Digital Preservation System':
    summary_df, individual_system_usage_df = calculate_system_usage(df, digi_pres_sys_cols, 'No digital preservation system')
    all_sys_fig = px.bar(summary_df, x='Usage', y='Count', text='Percentage',
              labels={'Usage': 'Digital Preservation System', 'Count': 'Count'},
              title='Percentage of Respondents Using<br>a Digital Preservation System',
              color='Usage',
              color_discrete_map={'Yes':'#3d96ac',
                                  'No':'#ac3d5f'})
    all_sys_fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    all_sys_fig.layout.update(uniformtext_minsize=10, uniformtext_mode='hide',
                    xaxis_title='System Usage', yaxis_title='Number of Respondants',
                    template='plotly_white')

    DPS_usage_per_cat = system_usage_perc(DPS_count, selected_category, digi_pres_sys_cols, 'No digital preservation system')
    DPS_usage_per_cat = DPS_usage_per_cat.sort_values(by='Any system %', ascending=True)
    DPS_usage_per_cat = DPS_usage_per_cat.reset_index()
    melted_dms = DPS_usage_per_cat.melt(id_vars=[selected_category], value_vars=['Any system %', 'No system %'], var_name='Digital Preservation System', value_name='Digital Preservation System Percentage')

    sys_fig = px.bar(melted_dms,
                            y=selected_category,
                            x='Digital Preservation System Percentage',
                            color='Digital Preservation System',
                            color_discrete_map={
                                'Any system %': '#3d96ac',
                                'No system %': '#ac3d5f'},
                            text='Digital Preservation System Percentage',
                            barmode='stack',
                            orientation='h',
                            height=650,width=1000,
                            labels={selected_category: selected_category, 'Digital Preservation System': 'Digital Preservation System'},
                            title=f'Digital Preservation System<br>Usage per {selected_category}')

    sys_fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')
    sys_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                                xaxis_title='Percentage', yaxis_title=f'Archive {selected_category}',
                                template='plotly_white')

    digi_pres_g = digi_pres_overall.sort_values(by='percentage', ascending=True)
    cms_fig = px.bar(digi_pres_g , y=digi_pres_g .index, x='percentage', title='Percentage of Respondents Using Each System',height=700,width=1200,
                    labels={'percentage': 'Percentage', 'count': 'Count'}, orientation='h', template='plotly_white' )
    cms_fig.update_traces(marker_color='#45a6bd')

    return hold_fig,stor_fig, com_fig, all_sys_fig, sys_fig, cms_fig

  elif selected_system == 'Digital Preservation Standard':
    summary_df, individual_system_usage_df = calculate_system_usage(df, digi_pres_tools_cols, 'No standard/guide used')
    all_sys_fig = px.bar(summary_df, x='Usage', y='Count', text='Percentage',
              labels={'Usage': 'Digital Preservation Standard', 'Count': 'Count'},
              title='Percentage of Respondents Using<br>a Digital Preservation Standard',
              color='Usage',
              color_discrete_map={'Yes':'#3d96ac',
                                  'No':'#ac3d5f'})
    all_sys_fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    all_sys_fig.layout.update(uniformtext_minsize=10, uniformtext_mode='hide',
                    xaxis_title='Standard Usage', yaxis_title='Number of Respondants',
                    template='plotly_white')

    standards_usage_per_cat = system_usage_perc(standards_count, selected_category, digi_pres_tools_cols, 'No standard/guide used')
    standards_usage_per_cat = standards_usage_per_cat.sort_values(by='Any system %', ascending=True)
    standards_usage_per_cat.columns = standards_usage_per_cat.columns.str.replace("system", "standard")
    standards_usage_per_cat = standards_usage_per_cat.reset_index()
    melted_standards = standards_usage_per_cat.melt(id_vars=[selected_category], value_vars=['Any standard %', 'No standard %'], var_name='Digital Preservation Standard', value_name='Digital Preservation Standard Percentage')
    sys_fig = px.bar(melted_standards,
                            y=selected_category,
                            x='Digital Preservation Standard Percentage',
                            color='Digital Preservation Standard',
                            color_discrete_map={
                                'Any standard %': '#3d96ac',
                                'No standard %': '#ac3d5f'},
                            text='Digital Preservation Standard Percentage',
                            barmode='stack',
                            orientation='h',
                            height=650,width=1000,
                            labels={selected_category: selected_category, 'Digital Preservation Standard': 'Digital Preservation Standard'},
                            title=f'Digital Preservation Standard<br>Usage per {selected_category}')

    sys_fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')
    sys_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                                xaxis_title='Percentage', yaxis_title=f'Archive {selected_category}',
                                template='plotly_white')

    digi_pres_tools_g = digi_pres_tools.sort_values(by='percentage', ascending=True)
    cms_fig = px.bar(digi_pres_tools_g, y=digi_pres_tools_g.index, x='percentage', title='Percentage of Respondents Using Each Standard', height=700,width=1400,
                 labels={'percentage': 'Percentage', 'count': 'Count'}, orientation='h', template='plotly_white' )
    cms_fig.update_traces(marker_color='#45a6bd')

    return hold_fig, stor_fig, com_fig, all_sys_fig, sys_fig, cms_fig



# Callback to update the graphs
@callback(
    [Output('category-graph', 'figure'),
    Output('overall-graph', 'figure'),
    Output('bin-graph', 'figure'),
    Output('volunteer-graph', 'figure')],
    [Input('category-selector', 'value'),
    Input('col-selector', 'value')]
)

def update_staff_graphs(selected_category, selected_job):
    # Calculate percentages by category using the selected job
    qualified_percentages_by_category = staff_percentages_by_category(df, selected_category, selected_job)
    qualified_percentages_by_category = qualified_percentages_by_category.sort_values(by='Percentage', ascending=False)

    # Plot the category graph
    category_fig = px.bar(qualified_percentages_by_category,
                          y=selected_category,
                          x='Percentage',
                          color=selected_job.replace('Total number of ', '').strip(),
                          color_discrete_map={
                              'Yes': '#4C78A8',
                              'None': 'goldenrod'},
                          text='Percentage',
                          barmode='stack',
                          orientation='h',
                          height=650, width=1000,
                          labels={selected_category: 'Category', 'Count': 'Count'},
                          title=f'{selected_job.replace("Total number of ", "").strip().title()} per Category')

    category_fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')
    category_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                               xaxis_title='Percentage', yaxis_title='Category',
                               template='plotly_white')

#     # Calculate overall percentages
#     overall_percentages = calc_zero_perc(df, selected_job)

#     # Plot the overall graph
#     overall_fig = px.bar(overall_percentages, x=overall_percentages.columns[0], y='Count', text='Percentage',height=600,color=overall_percentages.columns[0],
#                          color_discrete_map={f'Has {selected_job.replace("Total number of ", "").strip()}': '#4C78A8', f'No {selected_job.replace("Total number of ", "").strip()}': 'goldenrod'},title=f'Percentage of Archives with<br>{selected_job.replace("Total number of ", "").strip().title()}')

#     overall_fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
#     overall_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',xaxis_title= f'{selected_job.replace("Total number of ", "").strip()}', yaxis_title='Count',
# template='plotly_white')

    # Calculate overall percentages
    overall_percentages = calc_zero_perc_staff(df, selected_job)

    # Create the pie chart
    overall_fig = px.pie(
      overall_percentages,
        names=overall_percentages.columns[0],
      values='Count',
      color=overall_percentages.columns[0],
      color_discrete_map={
      f'Has {selected_job.replace("Total number of ", "").strip()}': '#4C78A8',
      f'No {selected_job.replace("Total number of ", "").strip()}': 'goldenrod'
      },
    title=f'Percentage of Archives with<br>{selected_job.replace("Total number of ", "").strip().title()}',
    hole=0.3  # This creates a donut chart
    )

  # Update the layout for the pie chart
    overall_fig.update_traces(textposition='inside', textinfo='percent')
    overall_fig.update_layout(
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    template='plotly_white',
    height=500,
    #width=500,
    margin=dict(l=10, r=40, t=120, b=20),  # Adjust margins to align chart
    legend=dict(
        orientation="h",  # Horizontal orientation
        yanchor="top",  # Position legend above the chart
        y=1.08,
        xanchor="center",
        x=0.5
    )
    )

    overall_fig.update_traces(
    hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent:.1%}'
    )


    # Plot binned data graph
    bin_fig = px.bar(archivistBins, x='archivists grouped', y='count',
             labels={'archivists grouped': 'archivists grouped', 'count': 'count'},
             title='Number of Qualified Archivists (FTE) Grouped')
    bin_fig.update_traces(marker_color='#4C78A8')
    bin_fig.update_layout(xaxis_title='Number of Qualified Archivists (FTE)', yaxis_title='Number of archives',
                  template='plotly_white')

    # Plot volunteer graph
    vol_df= volunteers_bar_chart(df, selected_category, vol_cols)

    vol_fig = px.bar(vol_df,
                     y=selected_category,
                     x='Count',
                     color='Volunteer Type',
                     barmode='stack',
                     orientation='h',
                     height=600,
                     color_discrete_map={
                              'Total number of onsite volunteers': '#4C78A8',
                              'Total number of online volunteers': 'rgb(136, 204, 238)'},
                     title='Onsite and Online Volunteers by {}'.format(selected_category))
    vol_fig.update_layout(yaxis={'categoryorder':'total ascending'}, margin=dict(l=30, r=30, t=100, b=20), template='plotly_white')


    return overall_fig, category_fig, bin_fig, vol_fig


@callback(
    [Output('cat-online-graph', 'figure'),
     Output('cat-unlist-graph', 'figure'),
     Output('cat-vs-graph', 'figure'),
     Output('opening-stats-graph', 'figure'),
     Output('vis-graph', 'figure')],
    Input('category-selector', 'value')
    )

def update_access_graphs(selected_category):

  cat_online = calc_zero_perc(df, 'Percentage of collection with a full catalogue available online')
  # cat_on_fig = px.pie(
  #   cat_online,
  #   names='percentage of collection with a full catalogue available online',
  #   values='Count',
  #   color='percentage of collection with a full catalogue available online',
  #   color_discrete_map={
  #   'Zero percentage of collection with a full catalogue available online': '#46a3d0',
  #   'Some percentage of collection with a full catalogue available online': '#d0465e'
  #   },
  #   title='Percentage of Archives with<br>Collections Catalogued Online',
  #   hole=0.3  # This creates a donut chart
  # )

  # # Update the layout
  # cat_on_fig.update_traces(textposition='inside', textinfo='percent')
  # cat_on_fig.update_layout(
  #   uniformtext_minsize=10,
  #   uniformtext_mode='hide',
  #   template='plotly_white',
  #   height=500,
  #   width=600,
  #   margin=dict(l=5, r=100, t=180, b=5),  # Adjust margins to align chart
  #   legend=dict(
  #   orientation="h",  # Horizontal orientation
  #   yanchor="top",  # Position legend above the chart
  #   y=1.2,
  #   xanchor="center",
  #   x=0.4
  # )
  # )
  # cat_on_fig.update_layout(legend_traceorder="reversed")
  # cat_on_fig.update_traces(
  # hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent:.1%}'
  # )

  cat_on_fig = px.bar(
  cat_online,
  x='percentage of collection with a full catalogue available online',
  y='Count',
  text='Percentage',
  color='percentage of collection with a full catalogue available online',
  color_discrete_map={
  'Zero percentage of collection with a full catalogue available online': '#46a3d0',
  'Some percentage of collection with a full catalogue available online': '#d0465e'
  },
  title='Percentage of Archives with<br>Collections Catalogued Online',
  labels={'percentage of collection with a full catalogue available online': 'Full catalogue', 'Percentage': 'Percentage'}
)

  # Update the layout
  cat_on_fig.update_traces(textposition='inside', texttemplate='%{text:.1f}%',)
  cat_on_fig.update_layout(
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    template='plotly_white',
    height=500,
    width=600,
    margin=dict(l=5, r=10, t=180, b=5),  # Adjust margins to align chart
    legend=dict(
    orientation="h",  # Horizontal orientation
    yanchor="top",  # Position legend above the chart
    y=1.2,
    xanchor="center",
    x=0.3
  )
  )
  cat_on_fig.update_layout(showlegend=False, xaxis_title='Percentage with full online catalogue', yaxis_title='Number of Archives')
  cat_on_fig.update_xaxes(tickvals=['Zero percentage of collection with a full catalogue available online', 'Some percentage of collection with a full catalogue available online', ], ticktext=['Zero percentage', 'Some percentage'])
  for trace in cat_on_fig.data:
    if trace.name == 'Zero percentage of collection with a full catalogue available online':
        trace.name = 'Zero percentage catalogue online'
    elif trace.name == 'Some percentage of collection with a full catalogue available online':
        trace.name = 'Some percentage catalogue online'

  cat_unlist = calc_zero_perc(df, 'Percentage of collection completely unlisted')
  # unlst_fig = px.pie(
  # cat_unlist,
  # names='percentage of collection completely unlisted',
  # values='Count',
  # color='percentage of collection completely unlisted',
  # color_discrete_map={
  # 'Zero percentage of collection completely unlisted': '#46a3d0',
  # 'Some percentage of collection completely unlisted': '#d0465e'
  # },
  # title='Percentage of Archives with<br>Collections Unlisted',
  # hole=0.3  # This creates a donut chart
  # )

  # # Update the layout for the pie chart
  # unlst_fig.update_traces(textposition='inside', textinfo='percent')
  # unlst_fig.update_layout(
  #   uniformtext_minsize=10,
  #   uniformtext_mode='hide',
  #   template='plotly_white',
  #   height=450,
  #   width=350,
  #   margin=dict(l=10, r=10, t=180, b=5),  # Adjust margins to align chart
  #   legend=dict(
  #   orientation="h",  # Horizontal orientation
  #   yanchor="top",  # Position legend above the chart
  #   y=1.2,
  #   xanchor="center",
  #   x=0.6
  # )
  # )
  # unlst_fig.update_layout(legend_traceorder="reversed")
  # unlst_fig.update_traces(
  # hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent:.1%}'
  # )
  unlst_fig = px.bar(
  cat_unlist,
  x='percentage of collection completely unlisted',
  y='Count',
  text='Percentage',
  color='percentage of collection completely unlisted',
  color_discrete_map={
  'Zero percentage of collection completely unlisted': '#46a3d0',
  'Some percentage of collection completely unlisted': '#d0465e'
  },
  title='Percentage of Archives with<br>Collections Unlisted',
  labels={'percentage of collection completely unlisted': 'Unlisted', 'Percentage': 'Percentage'}
  )
  # Update the layout
  unlst_fig.update_traces(textposition='inside', texttemplate='%{text:.1f}%',)
  unlst_fig.update_layout(
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    template='plotly_white',
    height=500,
    width=600,
    margin=dict(l=10, r=5, t=180, b=5),  # Adjust margins to align chart
    legend=dict(
    orientation="h",  # Horizontal orientation
    yanchor="top",  # Position legend above the chart
    y=1.2,
    xanchor="center",
    x=0.7
  )
  )
  unlst_fig.update_layout(showlegend=False, xaxis_title='Percentage unlisted', yaxis_title='Number of Archives')
  unlst_fig.update_xaxes(tickvals=['Zero percentage of collection completely unlisted', 'Some percentage of collection completely unlisted', ], ticktext=['Zero percentage', 'Some percentage'])
  for trace in unlst_fig.data:
    if trace.name == 'Zero percentage of collection completely unlisted':
        trace.name = 'Zero percentage completely unlisted'
    elif trace.name == 'Some percentage of collection completely unlisted':
        trace.name = 'Some percentage completely unlisted'


  unlist = df.groupby(selected_category)['Percentage of collection completely unlisted'].mean().round(2)
  unlist = unlist.to_frame().reset_index().sort_values(by='Percentage of collection completely unlisted', ascending=False)
  online = df.groupby(selected_category)['Percentage of collection with a full catalogue available online'].mean().round(2)
  online = online.to_frame().reset_index().sort_values(by='Percentage of collection with a full catalogue available online', ascending=False)
  cats = pd.merge(online, unlist, on=selected_category)
  cats = cats.sort_values(by=['Percentage of collection with a full catalogue available online','Percentage of collection completely unlisted'], ascending=[True, False])

  cat_vs_fig = px.bar(
  cats,
  y=selected_category,x=["Percentage of collection with a full catalogue available online", "Percentage of collection completely unlisted"],
  opacity = 0.9,
  orientation="h",
  barmode='group',
  title=f'Average Percentage of Collections<br>Fully Catalogued Online vs Completely Unlisted Per {selected_category}',
  labels= {"value": 'Average percentage',
           'variable': ''
  },
  height=600
  )
  cat_vs_fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')
  cat_vs_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                              xaxis_title='Percentage', yaxis_title=f'Archive {selected_category}',
                              template='plotly_white', margin=dict(l=10, r=10, t=150, b=40),
                              legend=dict(
    orientation="h",  # Horizontal orientation
    yanchor="top",  # Position legend above the chart
    y=1.1,
    xanchor="center",
    x=0.4
  ))
  cat_vs_fig.update_traces(marker_color="#d0465e", selector={"name": "Percentage of collection with a full catalogue available online"})
  cat_vs_fig.update_traces(marker_color='#46a3d0', selector={"name": "Percentage of collection completely unlisted"})

# Recalculate opening stats based on the selected category
  opening_data = opening_stats(df, selected_category, 'Open to the public')
  # Melt the DataFrame for Plotly
  melted = opening_data.reset_index().melt(id_vars=[selected_category], value_vars=['Yes_percentage', 'Yes, but booking required_percentage',
                                                                                    'Yes, with conditions (e.g. letter of introduction)_percentage','No, archives are not accessible to the public_percentage'],
                                            var_name='Response', value_name='Percentage')

  # Replace the response labels for better readability
  melted['Response'] = melted['Response'].str.replace('_percentage', '')

  open_fig = px.bar(melted, y=selected_category, x='Percentage', color='Response', color_discrete_map={'Yes': '#8f2437',
                                                                                        'Yes, but booking required': '#c9334d',
                                                                                        'Yes, with conditions (e.g. letter of introduction)': '#da6e8a',
                                                                                        'No, archives are not accessible to the public': '#46a3d0'},

                text='Percentage', orientation='h',
                title=f'Archives Open to the Public by {selected_category}', labels={selected_category: selected_category, 'Percentage': 'Percentage', 'Response': 'Response'},
                height=800, width=1400, template='plotly_white')

  visitors = allData.groupby(selected_category)['Visitors to your archive service'].mean().round(2)
  visitors = visitors.to_frame().reset_index().rename(columns = {'Visitors to your archive service':'Average Visitor Numbers'})
  visitors = visitors.sort_values(by='Average Visitor Numbers', ascending=True)
  vis_fig = px.bar(visitors, y=selected_category, x='Average Visitor Numbers', orientation='h',
            labels={selected_category: f"Archive {selected_category}", 'Average Visitor Numbers': 'Average Visitor numbers'},
            title=f'Average Yearly Visitor Numbers per {selected_category}')
  vis_fig.update_traces(marker_color='#c9334d')
  vis_fig.update_layout(xaxis_title='Average Number of Visitors', yaxis_title=f'Archive {selected_category}',
                template='plotly_white')

  return cat_on_fig, unlst_fig, cat_vs_fig, open_fig, vis_fig


@callback(
    [Output('all-ex-graph', 'figure'),
     Output('ex-graph', 'figure'),
     Output('soc-graph', 'figure')
    ],
    [Input('category-selector', 'value'),
     Input('ex-learn-selector', 'value')
     ]
)
def update_ex_graphs(selected_category, selected_event):
  if selected_event == 'Exhibitions':
      cols = exhibition_cols
  else:
      cols = learning_cols

  #cols = exhibition_cols
  #exhibitions_perc = calc_exhibitions_perc(df, 'Category', cols)
  grouped_ex = df.groupby(selected_category)[cols].sum()
  grouped_ex = grouped_ex.sort_values(by=cols[0], ascending=True)

  ex_fig = px.bar(grouped_ex,
                  y=grouped_ex.index,
                  x=cols,
                  color_discrete_map={col: color for col, color in zip(cols, ['#bf72c1', '#72c1bf', '#c1bf72'])},
                  barmode='stack',
                  orientation='h',
                  labels={selected_category:selected_category, 'variable': 'Type', 'value': 'count'},
                  title=f'Number of {selected_event} per {selected_category}',
                  height=800, width=1100)

  ex_fig.update_layout(
      uniformtext_minsize=10,
      uniformtext_mode='hide',
      xaxis_title=f'Number of {selected_event}',
      yaxis_title=f'Archive {selected_category}',
      template='plotly_white',
      legend=dict(
          orientation="v",
          yanchor="bottom",
          y=0.5,
          xanchor="right",
          x=0.9
      )
  )

  exhibitions_overall = _overall_perc(df, cols)
  exhibitions_overall=exhibitions_overall.reset_index().rename(columns={"index": f"{selected_event} type"})
  # all_ex_fig = px.pie(
  # exhibitions_overall,
  # names=f"{selected_event} type",
  # values='count',
  # color=f"{selected_event} type",
  # color_discrete_map={col: color for col, color in zip(cols, ['#bf72c1', '#72c1bf', '#c1bf72'])},
  # title=f'Percentage of {selected_event} Per Type',
  # hole=0.3  # This creates a donut chart
  # )

  # # Update the layout for the pie chart
  # all_ex_fig.update_traces(textposition='inside', textinfo='percent')
  # all_ex_fig.update_layout(
  #   uniformtext_minsize=10,
  #   uniformtext_mode='hide',
  #   template='plotly_white',
  #   height=450,
  #   width=650,
  #   margin=dict(l=20, r=20, t=40, b=5),  # Adjust margins to align chart
  #   legend=dict(
  #   orientation="v",  # Horizontal orientation
  #   yanchor="middle",  # Position legend above the chart
  #   y=0.5,
  #   xanchor="right",
  #   x=4.0
  # )
  # )
  # all_ex_fig.update_layout(legend_traceorder="reversed")
  # all_ex_fig.update_traces(
  # hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent:.1%}'
  # )

  all_ex_fig = px.bar(
    exhibitions_overall,
    x=f"{selected_event} type",
    y='count',
    text='percentage',
    color=f"{selected_event} type",
    color_discrete_map={col: color for col, color in zip(cols, ['#bf72c1', '#72c1bf', '#c1bf72'])},
    labels={'Count':f'Number of {selected_event}','percentage':'Percentage'},
    title=f'Percentage of {selected_event} Per Type'
  )
  if selected_event == 'Exhibitions':
    all_ex_fig.update_xaxes(tickvals=[f'{selected_event} held online', f'{selected_event}/displays held at external venues',f'{selected_event}/displays held in-house' ], ticktext=['held online', ' held at external venues', 'held in-house'])
  else:
    all_ex_fig.update_xaxes(tickvals=[f'{selected_event} held online', f'{selected_event} held at external venues',f'{selected_event} held in-house' ], ticktext=['held online', ' held at external venues', 'held in-house'])

  # Update the layout for the bar chart
  all_ex_fig.update_layout(
      uniformtext_minsize=10,
      uniformtext_mode='hide',
      template='plotly_white',
      height=450,
      width=950,
      yaxis_title=f'Number of {selected_event}',
      xaxis_title=f'{selected_event} type',
      margin=dict(l=20, r=20, t=40, b=5),
      legend=dict(
          orientation="v",
          yanchor="middle",
          y=0.5,
          xanchor="right",
          x=4.0
      ),
  )
  tickvals = exhibitions_overall[f"{selected_event} type"].unique()
  ticktext = ['held in-house', 'held at external venues', 'held online' ]

# Update x-axis based on selected_event
  all_ex_fig.update_xaxes(
    tickvals=tickvals,
    ticktext=ticktext)
  all_ex_fig.update_traces(textposition='inside', texttemplate='%{text:.1f}%',)

  soc_med2 = allData.groupby(selected_category)[sm_cols].sum()
  nan_value = float('NaN')
  soc_med2.replace('', nan_value, inplace=True)
  soc_med2.dropna(inplace=True)
  soc_med2 = soc_med2.reset_index()

  melt_sm = pd.melt(soc_med2, id_vars=[selected_category],
                      value_vars=sm_cols,
                      var_name='Platform',
                      value_name='Followers')
  melt_sm = melt_sm.sort_values(['Platform', 'Followers'], ascending=[False,True])

  min_size = 1
  melt_sm = melt_sm[melt_sm['Followers'] > 0]
  melt_sm['Followers'] = melt_sm['Followers'] + min_size

  # Create a treemap
  soc_fig = px.treemap(melt_sm,
                  path=[selected_category, 'Platform'],
                  values='Followers',
                  color='Followers',
                  color_continuous_scale='Magenta',
                  title=f'Treemap of Social Media Followers by {selected_category} and Platform',
                  labels={'labels':'Social Media Platform', 'Followers_sum': 'Number of Followers', 'parent': selected_category},
                  custom_data=['Platform', 'Followers', selected_category])
  soc_fig.update_traces(
      hovertemplate='<b>%{label}</b><br>Platform: %{customdata[0]}<br>Followers: %{customdata[1]}<br>Category: %{customdata[2]}<extra></extra>'
  )
  soc_fig.update_layout(height=800, width=1100, template='plotly_white')

  return all_ex_fig, ex_fig, soc_fig


# Callback to update the graphs
@callback(
    [Output('map-graph', 'figure'),
     Output('fin-graph', 'figure'),
     Output('fin-category-graph', 'figure'),
     Output('zero-fin-graph', 'figure')],
    Input('f-selector', 'value')
)
def update_fin_graphs(selected_job):
  counts = df['Postcode Region'].value_counts().to_frame('No. of respondants to survey')
  df1 = df.groupby('Postcode Region')[selected_job].sum().round(2).reset_index()
  df1 = counts.merge(df1, on='Postcode Region')
  merged_gdfr = gdf_regions.merge(df1, left_on='nuts118nm', right_on='Postcode Region')
  merged_gdfr = merged_gdfr.to_crs("WGS84")
  merged_gdfr.set_index('nuts118nm', inplace=True)

  map_fig = px.choropleth(
    merged_gdfr,
    geojson=merged_gdfr.geometry,
    locations=merged_gdfr.index,
    #featureidkey="properties.RGN23CD",  # Adjust this to match the GeoJSON properties key
    color=selected_job,
    color_continuous_scale="Ylgn",
    #range_color=(merged_gdfr['Total income in the last financial year'].min(), merged_gdfr['Total income in the last financial year'].max()),
    labels={selected_job: selected_job, 'nuts118nm': 'Region'},
    hover_data = {#'Postcode Region': True,
                  'No. of respondants to survey': True,
                  selected_job: True},
    title=f'{selected_job} by region'
)

  # Update the layout to display the map
  map_fig.update_geos(fitbounds='geojson', visible=False)
  map_fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})


  fin_fig = px.bar(financeBins, x='income grouped', y='count',
             labels={'income grouped': '	Income grouped', 'count': 'Count'},
             title='Total Income (£) Grouped')
  fin_fig.update_layout(xaxis_title='	Income grouped', yaxis_title='Number of archives',
                  template='plotly_white',
                  width=1000, height=600)
  fin_fig.update_traces(marker_color='rgb(102,166,30)')


    # Calculate overall percentages
  zero_percentages = calc_fzero_perc(df, selected_job)
    # Plot the overall graph
  zero_fig = px.bar(zero_percentages,
                        x=zero_percentages.columns[0],
                        y='Count',
                        text='Percentage',
                        height=600,
                        color_discrete_map={
                             f'Some {selected_job.replace(" in the last financial year", "").strip().lower()}': 'rgb(115, 175, 72)',
                             f'Zero {selected_job.replace(" in the last financial year", "").strip().lower()}': 'rgb(225, 124, 5)'},
                         color=zero_percentages.columns[0],
                         title=f'Percentage of Archives with<br>{selected_job.title()}')

  zero_fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
  zero_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                              xaxis_title='', yaxis_title='Number of Archives',
                              template='plotly_white')
  zero_fig.update_xaxes(showticklabels=False)

  df1 = df.groupby('Category')[selected_job].sum().round(2).reset_index()
  df1 = df1.sort_values(by=selected_job, ascending=True)
  fcat_fig = px.bar(df1, x=selected_job, y='Category',text=selected_job,
             labels={selected_job: f'{selected_job.replace("in the last financial year", "").strip()}'},
             title=f'{selected_job.replace("in the last financial year", "").strip().title()} Per Category')
  fcat_fig.update_traces(texttemplate='%{text:.3s}', textposition='none', marker_color='rgb(166,216,84)')
  fcat_fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide',
                  #width=1100,
                  xaxis_title=f'{selected_job.replace("in the last financial year", "").strip()}', yaxis_title='Category',
                  template='plotly_white')



  # Show the graphs
  return map_fig, fin_fig, zero_fig, fcat_fig


if __name__ == '__main__':
    app.run(jupyter_mode='external')
    #app.run(debug=True)
